# Katas del módulo 10

# Tracebacks

Si intentamos en un notebook, abrir un archivo inexistente sucede lo siguiente:

In [1]:
open("/path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

Vamos a crear un archivo open.py con el siguiente código.

In [ ]:
def main():
    open("/path/to/mars.jpg")


if __name__ == '__main__':
    main()

Vamos a ejecutar el código y visualizar el mensaje de error.

In [8]:
!python open.py

Traceback (most recent call last):
  File "c:\Users\lucy_\OneDrive\Escritorio\LaunchX-OnBoarding\katas\module-10\open.py", line 6, in <module>
    main()
  File "c:\Users\lucy_\OneDrive\Escritorio\LaunchX-OnBoarding\katas\module-10\open.py", line 2, in main
    open("/path/to/mars.jpg")
FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'


# Controlando las excepciones

## Try y Except de los bloques

Sabemos que, si no existe un archivo o directorio, se genera FileNotFoundError. Si queremos controlar esa excepción, podemos hacerlo con un bloque try y except.

A continuación se presente un ejemplo de uso.

In [3]:
try:
    open('config.txt')
except FileNotFoundError:
    print("Couldn't find the config.txt file!")

Couldn't find the config.txt file!


Vamos a crear un archivo de Python denominado *config.py*. El archivo tiene código que busca y lee el archivo de configuración del sistema de navegación.
El archivo *config.py* posee el siguiente código:

In [5]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")


if __name__ == '__main__':
    main()

A continuación, quitamos el archivo *config.txt* y creamos un directorio denominado *config.txt*. Intentaremos llamar al archivo *config.py* para ver un error nuevo que debería ser similar al siguiente:

In [31]:
!python config.py

Traceback (most recent call last):
  File "c:\Users\lucy_\OneDrive\Escritorio\LaunchX-OnBoarding\katas\module-10\config.py", line 9, in <module>
    main()
  File "c:\Users\lucy_\OneDrive\Escritorio\LaunchX-OnBoarding\katas\module-10\config.py", line 3, in main
    configuration = open('config.txt')
PermissionError: [Errno 13] Permission denied: 'config.txt'


Una manera poco útil de controlar este error sería detectar todas las excepciones posibles para evitar un traceback. 
Por lo que actualizamos la función *main()* en código del archivo *config.py* con el siguiente:

In [ ]:
def main():
    try:
        configuration = open('config.txt')
    except Exception:
        print("Couldn't find the config.txt file!")

Ahora volvemos a ejecutar el código en el mismo lugar donde existe el archivo config.txt con permisos incorrectos:

In [32]:
!python config.py

Couldn't find the config.txt file!


El archivo existe, pero tiene permisos diferentes y Python no puede leerlo. Cuando se trata con errores de software, puede resultar frustrante tener errores que hagan lo siguiente:

* No indiquen cuál es el problema real.
* Proporcionen una salida que no coincida con el problema real.
* No sugieran lo que se puede hacer para corregir el problema.

Vamos a corregir este fragmento de código para abordar todas estas frustraciones. 
Revertiremos la detección de FileNotFoundError y luego agregamos otro bloque except para detectar PermissionError:

In [ ]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except IsADirectoryError:
        print("Found config.txt but it is a directory, couldn't read it")
    except PermissionError:
        print("Found config.txt but couldn't read it")


Ahora volvemos a ejecutarlo, en el mismo lugar donde config.txt está con el problema de permisos:

In [33]:
!python config.py

Found config.txt but couldn't read it


Eliminamos el archivo config.txt para asegurarnos de que se alcanza el primer bloque except en su lugar:

In [39]:
!python config.py

Couldn't find the config.txt file!


Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea except. Por ejemplo, si el sistema de navegación está bajo cargas pesadas y el sistema de archivos está demasiado ocupado, tiene sentido detectar BlockingIOError y TimeOutError juntos:

In [ ]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except IsADirectoryError:
        print("Found config.txt but it is a directory, couldn't read it")
    except (BlockingIOError, TimeoutError):
        print("Filesystem under heavy load, can't complete reading configuration file")

Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil:



In [40]:
try:
    open("mars.jpg")
except FileNotFoundError as err:
    print("got a problem trying to read the file:", err)

got a problem trying to read the file: [Errno 2] No such file or directory: 'mars.jpg'


En este caso, as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. Otra razón para usar esta técnica es acceder directamente a los atributos del error. Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno:

In [41]:
try:
    open("config.txt")
except OSError as err:
    if err.errno == 2:
        print("Couldn't find the config.txt file!")
    elif err.errno == 13:
        print("Found config.txt but couldn't read it")

Couldn't find the config.txt file!


## Generación de excepciones

Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:


In [43]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Probemos con cinco astronautas, 100 litros de agua sobrante y dos días:

In [44]:
water_left(5, 100, 2)

'Total water left after 2 days is: -10 liters'

Esto no es muy útil, ya que una carencia en los litros sería un error. Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error:

In [46]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(
            f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Ahora volvemos a ejecutarlo

In [47]:
water_left(5, 100, 2)

RuntimeError: There is not enough water for 5 astronauts after 2 days!

En el sistema de navegación, el código para señalar la alerta ahora puede usar RuntimeError para generar la alerta:

In [48]:
try:
    water_left(5, 100, 2)
except RuntimeError as err:
    alert_navigation_system(err)

NameError: name 'alert_navigation_system' is not defined

La función water_left() también se puede actualizar para evitar el paso de tipos no admitidos. Intentenis pasar argumentos que no sean enteros para comprobar la salida de error:

In [49]:
water_left("3", "200", None)

TypeError: can't multiply sequence by non-int of type 'NoneType'

El error de TypeError no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use TypeError, pero con un mensaje mejor:

In [51]:
def water_left(astronauts, water_left, days_left):
    for argument in [astronauts, water_left, days_left]:
        try:
            # If argument is an int, the following operation will work
            argument / 10
        except TypeError:
            # TypError will be raised only if it isn't the right type
            # Raise the same exception but with a better error message
            raise TypeError(
                f"All arguments must be of type int, but received: '{argument}'")
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(
            f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Ahora volvemos a intentarlo para obtener un error mejor:

In [52]:
water_left("3", "200", None)

TypeError: All arguments must be of type int, but received: '3'